# **Atmospheric correction of Sentinel 2 image using Py6S in Google Colab environment**

This is the second part of python codes used in the article. The codes are tested inside Google Colab environment using Hong Kong water as the study area.

**Import required libraries & Initialize Google Earth Engine session**

In [ ]:
import ee
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import statsmodels.api as sm
ee.Authenticate()
ee.Initialize()

# Step 1 - Match image data & in-situ station data

In [ ]:
# Load image data & in-situ station data
assetList = ee.data.getList({'id':"users/khoyinivan/S2_Py6S_mask_m"})
url = 'https://raw.githubusercontent.com/ivanhykwong/Marine-Water-Quality-Time-Series-HK/main/MarineQuality_2015-2020.csv'
station_list = ['TM2','TM3','TM4','TM5','TM6','TM7','TM8','SM1','SM2','SM3','SM4','SM5','SM6','SM7','SM9','SM10','SM11',
                'SM12','SM13','SM17','SM18','SM19','SM20','PM1','PM2','PM3','PM4','PM6','PM7','PM8','PM9','PM11','JM3',
                'JM4','DM1','DM2','DM3','DM4','DM5','NM1','NM2','NM3','NM5','NM6','NM8','MM1','MM2','MM3','MM4','MM5',
                'MM6','MM7','MM8','MM13','MM14','MM15','MM16','MM17','MM19','WM1','WM2','WM3','WM4','EM1','EM2','EM3',
                'VM1','VM2','VM4','VM5','VM6','VM7','VM8','VM12','VM14','VM15']
df_url = pd.read_csv(url)
df_url = df_url[df_url['Station'].isin(station_list)]
print(assetList)
print(len(assetList))
aoi = ee.Geometry.Polygon([[[113.800, 22.570],[113.800, 22.120],[114.514, 22.120],[114.514, 22.570]]])
df_data = pd.DataFrame()

for i in range(len(assetList)):
  # Extract image date
  assetid = assetList[i]['id']
  print(assetid)
  d1 = ee.Image(assetid)
  d1_date = d1.date().format('yyyy-MM-dd')
  print(d1_date.getInfo())

  # Find nearest date between image & station data
  df = df_url.copy()
  df['Dates'] = pd.to_datetime(df['Dates'], format='%Y-%m-%d')
  imagedate = datetime.strptime(d1_date.getInfo(), '%Y-%m-%d')
  df['Image_date'] = imagedate
  df['Date_compare'] = abs(df['Dates'] - imagedate)
  df = df.sort_values(by=['Date_compare'])
  df = df.drop_duplicates(subset=['Station'])
  print(df.shape)

  # Match image & station data, extract values to dataframe
  pts = ee.FeatureCollection("users/khoyinivan/MonitoringStation_wgs84_76")
  pt_list = pts.toList(pts.size())
  df[['B1','B2','B3','B4','B5','B6','B7','B8','B11','B12']] = np.nan
  for pt in range(pt_list.length().getInfo()):
    pt1 = ee.Feature(pt_list.get(pt))
    pt1_buf = pt1.buffer(20)
    s2_dict = d1.reduceRegion(ee.Reducer.mean(), pt1_buf.geometry()).getInfo()
    n = pt1_buf.getInfo()['properties']['WaterStati']
    for b in ['B1','B2','B3','B4','B5','B6','B7','B8','B11','B12']:
      df.loc[df['Station'] == n, b] = s2_dict[b]
  df = df.dropna(subset = ['B2'])
  df['n'] = df.shape[0]

  # Combine all image dates
  df_data = pd.concat([df_data, df])

# Export tables
df_data.to_csv('df_data.csv')

This block is written for inputing csv file as the data, skip this block if the dataframe is already loaded

In [ ]:
# for start from reading file only, skip this block if from above
df_data = pd.read_csv('df_data.csv')
df_data['Image_date'] = pd.to_datetime(df_data['Image_date'], format='%Y-%m-%d')
df_data['Date_compare'] = pd.to_timedelta(df_data['Date_compare'])
df_data = df_data.drop(columns=['Unnamed: 0'])


# Step 2 - Extract observations & create variables

In [ ]:
# Extract observations with <1 day difference

max_day_diff = 1

df = df_data[['Image_date', 'Dates', 'Date_compare', 'n', 
              '5-day Biochemical Oxygen Demand mg_L', 'Ammonia Nitrogen mg_L', 'Chlorophyll-a ug_L', 'Dissolved Oxygen mg_L',
              'E. coli cfu_100mL', 'Faecal Coliforms cfu_100mL', 'Nitrate Nitrogen mg_L', 'Nitrite Nitrogen mg_L', 
              'Orthophosphate Phosphorus mg_L', 'pH', 'Salinity psu', 'Secchi Disc Depth M', 'Silica mg_L', 
              'Suspended Solids mg_L', 'Temperature C', 'Total Inorganic Nitrogen mg_L', 'Total Kjeldahl Nitrogen mg_L', 
              'Total Nitrogen mg_L', 'Total Phosphorus mg_L', 'Turbidity NTU', 'Unionised Ammonia mg_L', 'Volatile Suspended Solids mg_L',
              'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12']].copy()

df = df.rename(columns={'Image_date': 'Image_Date', 'Dates': 'Station_Date', 
                        '5-day Biochemical Oxygen Demand mg_L': 'BOD', 'Ammonia Nitrogen mg_L': 'AmNi', 'Chlorophyll-a ug_L': 'Chla', 'Dissolved Oxygen mg_L': 'DO',
                        'E. coli cfu_100mL': 'Ecoli', 'Faecal Coliforms cfu_100mL': 'FC', 'Nitrate Nitrogen mg_L': 'NitraNi', 'Nitrite Nitrogen mg_L': 'NitriNi', 
                        'Orthophosphate Phosphorus mg_L': 'OrPh', 'pH': 'pH', 'Salinity psu': 'Sal', 'Secchi Disc Depth M': 'SDD', 'Silica mg_L': 'Si', 
                        'Suspended Solids mg_L': 'SS', 'Temperature C': 'Temp', 'Total Inorganic Nitrogen mg_L': 'TIN', 'Total Kjeldahl Nitrogen mg_L': 'TKN', 
                        'Total Nitrogen mg_L': 'ToNi', 'Total Phosphorus mg_L': 'ToPh', 'Turbidity NTU': 'Tur', 'Unionised Ammonia mg_L': 'UnAm', 'Volatile Suspended Solids mg_L': 'VSS'})

df['Date_compare'] = pd.to_numeric(df['Date_compare'].dt.days)
df['Image_Year'] = pd.DatetimeIndex(df['Image_Date']).year

df = df[(df['Date_compare'] <= max_day_diff) & (df['n'] >= 10)].copy().drop(columns=['Image_Date', 'Station_Date', 'Date_compare', 'n'])

# Remove outlier using Tukey’s fences method

Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR)))[['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12']].any(axis=1)]

# Replace 0 to min/2 (avoid errors in log operation)

wq = ['BOD', 'AmNi', 'Chla', 'DO', 'Ecoli', 'FC', 'NitraNi', 'NitriNi', 'OrPh', 'pH', 'Sal', 'SDD', 'Si', 'SS', 'Temp', 'TIN', 'TKN', 'ToNi', 'ToPh', 'Tur', 'UnAm', 'VSS']
for a in wq:
  df[a]=df[a].replace(0, np.NaN)
  df[a]=df[a].replace(np.NaN,df[a].min()/2)
df


In [ ]:
# Create independent variables

bands = ['B' + str(b) for b in [*range(1,9),11,12]]
wl = [443,490,560,665,705,740,783,842,1610,2190]

# Two-band ratio
for i in bands:
  for j in bands:
    if i != j:
      df['BR_'+i+j] = df[i] / df[j]
      # B_ratio.append('B'+str(i)+'_B'+str(j))

# Three-band ratio
for i in range(0,10):
  for j in range(0,10):
    for k in range(0,10):
      if (j == i+1) & (k == j+1):
        df['TB_'+bands[i]+bands[j]+bands[k]] = ((1/df[bands[i]]) - (1/df[bands[j]])) * df[bands[k]]

# Line height algorithm
for i in range(0,10):
  for j in range(0,10):
    for k in range(0,10):
      if (j == i+1) & (k == j+1):
        df['LH_'+bands[i]+bands[j]+bands[k]] = df[bands[j]] - df[bands[i]] - ((df[bands[k]] - df[bands[i]]) * ((wl[j]-wl[i])/(wl[k]-wl[i])))

df.to_csv('df_data_filter.csv')


# Step 3 - Stepwise regression

In [ ]:
# Define functions for stepwise regression 
# Modified from https://datascience.stackexchange.com/questions/24405/how-to-do-stepwise-regression-using-sklearn/24447#24447

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.1, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - pandas.DataFrame with the target column
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    y = y.to_numpy()
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype='float64')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.index[pvalues.argmax()]
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

def fit_regression(df, X_all, Y_name):
  Y_ln = np.log(df[Y_name])
  
  stepwise_variables = stepwise_selection(X_all, Y_ln, verbose=False)
  # print('resulting features:')
  # print(stepwise_variables)

  X = X_all[stepwise_variables]
  regr = linear_model.LinearRegression()
  regr.fit(X, Y_ln)
  r_squared = regr.score(X, Y_ln)
  adjusted_r_squared = 1 - (1-r_squared)*(len(Y_ln)-1)/(len(Y_ln)-X.shape[1]-1)

  print(Y_name + ': ' + str(r_squared))

  # Evaluate model
  Y = df[Y_name]
  Y_pred = np.exp(regr.predict(X))
  corr_orig = np.corrcoef(Y, Y_pred)[0, 1]
  regr_eval = linear_model.LinearRegression()
  regr_eval.fit(Y_pred.reshape(-1, 1), Y)
  r_squared_orig = regr_eval.score(Y_pred.reshape(-1, 1), Y)
  adjusted_r_squared_orig = 1 - (1-r_squared_orig)*(len(Y)-1)/(len(Y)-X.shape[1]-1)
  rmse = mean_squared_error(Y, Y_pred, squared=False)
  rmspe = np.sqrt(np.mean(np.square(((Y - Y_pred) / Y)), axis=0))
  mae = mean_absolute_error(Y, Y_pred)
  mape = np.mean(np.abs(((Y - Y_pred) / Y)))

  regression_df = pd.DataFrame({'Y': [Y_name], 'r2': [r_squared], 'adjusted_r2': [adjusted_r_squared],
                                'corr_orig': [corr_orig], 'r2_orig': [r_squared_orig], 'adjusted_r2_orig': [adjusted_r_squared_orig],
                                'rmse': [rmse], 'rmspe': [rmspe], 'mae': [mae], 'mape': [mape], 
                                'X': [stepwise_variables], 'intercept': [regr.intercept_], 'coef': [regr.coef_]})
  predicted_df = pd.DataFrame({'Y_ln': Y_ln, 'Pred_ln': regr.predict(X), 'Y': Y, 'Pred': Y_pred})

  return regression_df, predicted_df

def test_regression(df_test, X_test, Y_name, regression_df):

  regr_int = regression_df['intercept'][0]
  regr_variables = regression_df['X'][0]
  regr_coef = regression_df['coef'][0]

  X_test['Y_pred'] = regr_int
  for n in range(len(regr_variables)):
    X_test['Y_pred'] = X_test['Y_pred'] + X_test[regr_variables[n]]*regr_coef[n]

  Y = df_test[Y_name]
  Y_ln = np.log(df_test[Y_name])
  Y_pred_ln = X_test['Y_pred']
  Y_pred = np.exp(X_test['Y_pred'])
  
  # Evaluate model
  regr = linear_model.LinearRegression()
  regr.fit(Y.values.reshape(-1,1), Y_pred.values.reshape(-1,1))
  r_squared = regr.score(Y.values.reshape(-1,1), Y_pred.values.reshape(-1,1))
  corr = np.corrcoef(Y, Y_pred)[0, 1]
  rmse = mean_squared_error(Y, Y_pred, squared=False)
  rmspe = np.sqrt(np.mean(np.square(((Y - Y_pred) / Y)), axis=0))
  mae = mean_absolute_error(Y, Y_pred)
  mape = np.mean(np.abs(((Y - Y_pred) / Y)))

  regr_ln = linear_model.LinearRegression()
  regr_ln.fit(Y_ln.values.reshape(-1,1), Y_pred_ln.values.reshape(-1,1))
  r_squared_ln = regr_ln.score(Y_ln.values.reshape(-1,1), Y_pred_ln.values.reshape(-1,1))
  corr_ln = np.corrcoef(Y_ln, Y_pred_ln)[0, 1]

  test_summary_df = pd.DataFrame({'Y': [Y_name], 'r2': [r_squared], 'r2_ln': [r_squared_ln],
                                'corr': [corr], 'corr_ln': [corr_ln], 
                                'rmse': [rmse], 'rmspe': [rmspe], 'mae': [mae], 'mape': [mape]})
  test_data_df = pd.DataFrame({'Y_ln': Y_ln, 'Y_pred_ln': Y_pred_ln, 'Y': Y, 'Y_pred': Y_pred})

  return test_summary_df, test_data_df


This block is written for inputing csv file as the data, skip this block if the dataframe is already loaded

In [ ]:
# for start from reading file
df = pd.read_csv('df_data_filter.csv')
df = df.drop(columns=['Unnamed: 0'])
df

In [ ]:
# Seperate into training/testing sets

df_train = df[df['Image_Year'] <= 2019].drop(columns=['Image_Year']).copy()
df_test = df[df['Image_Year'] == 2020].drop(columns=['Image_Year']).copy()
X_train = df_train.drop(columns = wq)
X_test = df_test.drop(columns = wq)

# Perform stepwise regression (model development)

train_regr_result_list, train_pred_result_list = map(list,zip(*[fit_regression(df_train, X_train, value) for value in wq]))
train_regr_result = pd.concat(train_regr_result_list)
train_pred_result = pd.concat(train_pred_result_list, axis=1)
wq_col_list = [[value+'_ln', value+'Pred_ln', value, value+'Pred'] for value in wq]
wq_col = [item for sublist in wq_col_list for item in sublist]
train_pred_result.columns = wq_col
train_regr_result


In [ ]:
# Perform stepwise regression (validation)

test_regr_summary_list, test_regr_data_list = map(list,zip(*[test_regression(df_test, X_test, value, train_regr_result[train_regr_result['Y']==value]) for value in wq]))

test_regr_summary = pd.concat(test_regr_summary_list)
test_regr_data = pd.concat(test_regr_data_list, axis=1)
test_regr_data.columns = wq_col
test_regr_summary



In [ ]:
# Save results

train_regr_result.to_csv('train_regr_result.csv')
train_pred_result.to_csv('train_pred_result.csv')
test_regr_summary.to_csv('test_regr_summary.csv')
test_regr_data.to_csv('test_regr_data.csv')


# Step 4 - Apply regression models

In [ ]:
# Apply regression models to images (entire image collection)

assetList = ee.data.getList({'id':"users/khoyinivan/S2_Py6S_mask_m"})
print(assetList)
print(len(assetList))
aoi = ee.Geometry.Polygon([[[113.800, 22.570],[113.800, 22.120],[114.514, 22.120],[114.514, 22.570]]])

for i in range(len(assetList)):
  assetid = assetList[i]['id']
  print(assetid)
  d1 = ee.Image(assetid)
  d1_date = d1.date().format('yyyy-MM-dd')
  print(d1_date.getInfo())
  imagedate = datetime.strptime(d1_date.getInfo(), '%Y-%m-%d')

  # Chla
  name = ('Chla' + d1_date.getInfo()).replace('-','')
  regr_int = 0.66220
  # Variable: 'LH_B1B2B3', 'B1', 'B5', 'LH_B7B8B11', 'LH_B8B11B12', 'BR_B3B2', 'BR_B12B3', 'LH_B2B3B4'
  regr_coef = [-137.59036, -14.78696, 15.69912, 55.41826, -25.37227, 1.18410, -0.42734, -18.73902]
  d1_predict = d1.expression(
      'Int + C0*(B2-B1-(B3-B1)*((490-443)/(560-443))) + C1*B1 + C2*B5 + C3*(B8-B7-(B11-B7)*((842-783)/(1610-783))) + C4*(B11-B8-(B12-B8)*((1610-842)/(2190-842))) + C5*(B3/B2) + C6*(B12/B3) + C7*(B3-B2-(B4-B2)*((560-490)/(665-490)))', {
          'Int': regr_int,
          'C0': regr_coef[0], 'C1': regr_coef[1], 'C2': regr_coef[2], 'C3': regr_coef[3],
          'C4': regr_coef[4], 'C5': regr_coef[5], 'C6': regr_coef[6], 'C7': regr_coef[7],
          'B1': d1.select('B1'), 'B2': d1.select('B2'), 'B3': d1.select('B3'), 'B4': d1.select('B4'), 'B5': d1.select('B5'), 
          'B7': d1.select('B7'), 'B8': d1.select('B8'), 'B11': d1.select('B11'), 'B12': d1.select('B12')
          }).exp().rename(name).set('system:time_start', ee.Date(d1_date).millis())

  task = ee.batch.Export.image.toAsset(image=d1_predict, description=name, assetId = 'users/khoyinivan/S2_Chla/' + name, scale = 10, region = aoi)
  task.start()

  # SS
  name = ('SS' + d1_date.getInfo()).replace('-','')
  regr_int = 0.80715
  # Variable: 'B3', 'LH_B4B5B6', 'LH_B8B11B12', 'BR_B12B3', 'LH_B7B8B11', 'B2', 'BR_B8B3', 'BR_B7B2', 'BR_B11B2'
  regr_coef = [38.11636, 56.64875, -54.68850, 1.03707, -49.44436, -30.55160, 1.80722, -1.32230, -0.93927]
  d1_predict = d1.expression(
      'Int + C0*B3 + C1*(B5-B4-(B6-B4)*((705-665)/(740-665))) + C2*(B11-B8-(B12-B8)*((1610-842)/(2190-842))) + C3*(B12/B3) + C4*(B8-B7-(B11-B7)*((842-783)/(1610-783))) + C5*B2 + C6*(B8/B3) + C7*(B7/B2) + C8*(B11/B2)', {
          'Int': regr_int,
          'C0': regr_coef[0], 'C1': regr_coef[1], 'C2': regr_coef[2], 'C3': regr_coef[3],
          'C4': regr_coef[4], 'C5': regr_coef[5], 'C6': regr_coef[6], 'C7': regr_coef[7], 'C8': regr_coef[8], 
          'B2': d1.select('B2'), 'B3': d1.select('B3'), 'B4': d1.select('B4'), 'B5': d1.select('B5'), 'B6': d1.select('B6'), 
          'B7': d1.select('B7'), 'B8': d1.select('B8'), 'B11': d1.select('B11'), 'B12': d1.select('B12')
          }).exp().rename(name).set('system:time_start', ee.Date(d1_date).millis())

  task = ee.batch.Export.image.toAsset(image=d1_predict, description=name, assetId = 'users/khoyinivan/S2_SS/' + name, scale = 10, region = aoi)
  task.start()

  # Tur
  name = ('Tur' + d1_date.getInfo()).replace('-','')
  regr_int = 2.67180
  # Variable: 'BR_B3B4', 'LH_B2B3B4', 'BR_B8B2', 'BR_B11B7', 'BR_B1B3', 'BR_B6B2', 'BR_B3B2', 'LH_B4B5B6', 'LH_B3B4B5'
  regr_coef = [0.00528, 64.73459, -2.00839, -0.00082, -0.99431, 2.11603, -1.54066, 119.18667, 32.45938]
  d1_predict = d1.expression(
      'Int + C0*(B3/B4) + C1*(B3-B2-(B4-B2)*((560-490)/(665-490))) + C2*(B8/B2) + C3*(B11/B7) + C4*(B1/B3) + C5*(B6/B2) + C6*(B3/B2) + C7*(B5-B4-(B6-B4)*((705-665)/(740-665))) + C8*(B4-B3-(B5-B3)*((665-560)/(705-560)))', {
          'Int': regr_int,
          'C0': regr_coef[0], 'C1': regr_coef[1], 'C2': regr_coef[2], 'C3': regr_coef[3],
          'C4': regr_coef[4], 'C5': regr_coef[5], 'C6': regr_coef[6], 'C7': regr_coef[7], 'C8': regr_coef[8], 
          'B1': d1.select('B1'), 'B2': d1.select('B2'), 'B3': d1.select('B3'), 'B4': d1.select('B4'), 'B5': d1.select('B5'), 'B6': d1.select('B6'), 
          'B7': d1.select('B7'), 'B8': d1.select('B8'), 'B11': d1.select('B11')
          }).exp().rename(name).set('system:time_start', ee.Date(d1_date).millis())

  task = ee.batch.Export.image.toAsset(image=d1_predict, description=name, assetId = 'users/khoyinivan/S2_Tur/' + name, scale = 10, region = aoi)
  task.start()


